In [16]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn import tree
from xgboost import plot_tree
import pandas as pd
import numpy as np
import os
import wget
from pathlib import Path
import shutil
import gzip
import joblib
from xgboost import XGBClassifier
from mlciic import functions
from mlciic import metrics as met
from matplotlib import pyplot as plt

random_state=42
np.random.seed(random_state)


df_train = pd.read_csv('../../../data/EdgeIIot_train_dummies.csv', low_memory=False)
df_test = pd.read_csv('../../../data/EdgeIIot_test_dummies.csv', low_memory=False)

functions.display_information_dataframe(df_train,showCategoricals = True, showDetailsOnCategorical = True, showFullDetails = True)

#print(df_train["Attack_type"].value_counts())

df_train.drop(["Attack_label"], axis=1, inplace=True)
df_test.drop(["Attack_label"], axis=1, inplace=True)

features = [ col for col in df_train.columns if col not in ["Attack_label"]+["Attack_type"]] 


#for the SMOTE part, so it can fit in 16gb of RAM
df_before = df_train
df_attacks = df_train[df_train["Attack_type"] != "Normal"]

df_normal = df_train[df_train["Attack_type"] == "Normal"]
df_normal = shuffle(df_normal)
df_normal = df_normal[:100000]
df_train = pd.concat([df_attacks,df_normal])
df_train = shuffle(df_train)





#Encoding
le = LabelEncoder()
le.fit(df_train["Attack_type"].values)

X_train = df_train[features].values
y_train = df_train["Attack_type"].values
y_train = le.transform(y_train)

X_test = df_test[features].values
y_test = df_test["Attack_type"].values
y_test = le.transform(y_test)

standScaler = StandardScaler()
model_norm = standScaler.fit(X_train)

---
Lines: 1527737
Columns: 97 
Missing value or NaN: 0
---
Categorical columns: 
['Attack_type']

--- Details for categorical columns ---
Attack_type: 
['Normal' 'DDoS_UDP' 'Password' 'DDoS_TCP' 'Backdoor' 'DDoS_ICMP'
 'Port_Scanning' 'Vulnerability_scanner' 'SQL_injection' 'DDoS_HTTP'
 'Uploading' 'XSS' 'Ransomware' 'MITM' 'Fingerprinting']



,Data Type,Column Name,Unique Values
0,float64,arp.opcode,"[0.0, 2.0, 1.0]"
1,float64,arp.hw.size,"[0.0, 6.0]"
2,float64,icmp.checksum,"[0.0, 25274.0, 56236.0, 32598.0, 21835.0, 1449..."
3,float64,icmp.seq_le,"[0.0, 57101.0, 15950.0, 19413.0, 41521.0, 3247..."
4,float64,icmp.unused,[0.0]
5,float64,http.content_length,"[0.0, 277.0, 57.0, 1415.0, 36.0, 1.0, 1465.0, ..."
6,float64,http.response,"[0.0, 1.0]"
7,float64,http.tls_port,[0.0]
8,float64,tcp.ack,"[6.0, 0.0, 91101.0, 3531.0, 5.0, 303.0, 59.0, ..."
9,float64,tcp.ack_raw,"[2774423095.0, 0.0, 2371715782.0, 2371628212.0..."


In [17]:
df_new = pd.DataFrame(X_train,columns=features)
df_new["Attack_type"] = y_train

In [18]:
functions.display_information_dataframe(df_new,showCategoricals = True, showDetailsOnCategorical = True, showFullDetails = True)

---
Lines: 536515
Columns: 96 
Missing value or NaN: 0
---
Categorical columns: 
[]

--- Details for categorical columns ---


,Data Type,Column Name,Unique Values
0,float64,arp.opcode,"[0.0, 1.0, 2.0]"
1,float64,arp.hw.size,"[0.0, 6.0]"
2,float64,icmp.checksum,"[0.0, 39813.0, 10638.0, 3216.0, 65447.0, 25047..."
3,float64,icmp.seq_le,"[0.0, 29461.0, 27271.0, 26788.0, 20657.0, 8535..."
4,float64,icmp.unused,[0.0]
5,float64,http.content_length,"[0.0, 164.0, 36.0, 298.0, 277.0, 5.0, 57.0, 12..."
6,float64,http.response,"[0.0, 1.0]"
7,float64,http.tls_port,[0.0]
8,float64,tcp.ack,"[0.0, 1.0, 15.0, 2.0, 152201873.0, 56.0, 486.0..."
9,float64,tcp.ack_raw,"[0.0, 2069710094.0, 1614372990.0, 3475944524.0..."


In [19]:
def checkFields(row):
    return np.sum(row == 1) == 1

def applyVerification(df, lst):
    result = np.apply_along_axis(checkFields, 1, df[lst].values)
    print(result)
    counts = np.bincount(result.astype(int))
    print(counts)

    print("Number of True values: ", counts[1])
    print("Number of False values: ", counts[0])

    # Remove rows with False values
    df_filtered = df[result]

    return df_filtered

In [23]:
lst = ['http.request.method_0', 'http.request.method_1', 'http.request.method_2', 'http.request.method_3', 'http.request.method_4', 'http.request.method_5', 'http.request.method_6', 'http.request.method_7', 'http.request.method_8']

df_new = applyVerification(df_new, lst)

lst = ['http.referer_0', 'http.referer_1', 'http.referer_2', 'http.referer_3', 'http.referer_4']

df_new = applyVerification(df_new, lst)

lst = ['http.request.version_0', 'http.request.version_1', 'http.request.version_2', 'http.request.version_3', 'http.request.version_4','http.request.version_5','http.request.version_6','http.request.version_7','http.request.version_8','http.request.version_9','http.request.version_10','http.request.version_11','http.request.version_12']

df_new = applyVerification(df_new, lst)

lst = ['dns.qry.name.len_0', 'dns.qry.name.len_1', 'dns.qry.name.len_2', 'dns.qry.name.len_3', 'dns.qry.name.len_4', 'dns.qry.name.len_5', 'dns.qry.name.len_6', 'dns.qry.name.len_7', 'dns.qry.name.len_8']

df_new = applyVerification(df_new, lst)

lst = ['mqtt.conack.flags_0', 'mqtt.conack.flags_1', 'mqtt.conack.flags_2', 'mqtt.conack.flags_3', 'mqtt.conack.flags_4','mqtt.conack.flags_5','mqtt.conack.flags_6','mqtt.conack.flags_7','mqtt.conack.flags_8','mqtt.conack.flags_9','mqtt.conack.flags_10','mqtt.conack.flags_11','mqtt.conack.flags_12']

df_new = applyVerification(df_new, lst)

lst = ['mqtt.protoname_0', 'mqtt.protoname_1', 'mqtt.protoname_2']

df_new = applyVerification(df_new, lst)

lst = ['mqtt.topic_0', 'mqtt.topic_1', 'mqtt.topic_2']

df_new = applyVerification(df_new, lst)


[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
[ True  True  True ...  True  True  True]
[     0 536515]
Number of True values:  536515
Number of False values:  0
